In [1]:
# for vectorized operations
import numpy as np

# for dataframe manipulation
import pandas as pd

# for vizualizations
import matplotlib.pyplot as plt
import seaborn as sns

# for statistical calculations
import scipy.stats as stats

# for obtaining stock datasets
from pydataset import data

# for manipulation of time data
from datetime import date

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# filter out warnings
import warnings
warnings.filterwarnings('ignore')

# our own functions for accessing our sql database
from env import get_db_url, user, password, host

# our own acquire script
import acquire

# show all columns for df.head()
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)


# Tidy Data

### 1. Attendance Data

In [2]:
url = get_db_url('tidy_data')
sql = '''
SELECT *
FROM attendance
'''

attendance = pd.read_sql(sql, url)

In [3]:
df = attendance
df

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


#### Calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [4]:
# one observation = one student on a given day

In [5]:
df = df.melt(id_vars='Unnamed: 0')

In [6]:
df.head(3)

,Unnamed: 0,variable,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A


In [7]:
df.columns = ['student', 'date', 'attendance']

In [8]:
df.head(3)

,student,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A


In [9]:
df['attendance'] = df.attendance.map({'P': 1, 'A': 0, 'H': .5, 'T': .9})

In [10]:
df.head(3)

,student,date,attendance
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0


In [11]:
df.groupby(by='student').attendance.mean()

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance, dtype: float64

### 2. Coffee Levels

#### 2a. Read the coffee_levels table

In [12]:
url = get_db_url('tidy_data')
sql = '''
SELECT *
FROM coffee_levels
'''
coffee_levels = pd.read_sql(sql, url)

In [13]:
df = coffee_levels

In [14]:
df.head(3)

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279


#### 2b. Transform the data so that each carafe is in it's own column

In [15]:
df = df.pivot(index='hour', columns='coffee_carafe')
df.head(3)

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928

In [16]:
df = df.reset_index()

In [17]:
df.head(3)

hour coffee_amount                    
coffee_carafe                  x         y         z
0                8      0.816164  0.189297  0.999264
1                9      0.451018  0.521502  0.915990
2               10      0.843279  0.023163  0.144928

In [18]:
df.columns.names = [None, None]

In [19]:
df.head(3)

hour coffee_amount                    
                   x         y         z
0    8      0.816164  0.189297  0.999264
1    9      0.451018  0.521502  0.915990
2   10      0.843279  0.023163  0.144928

#### 2c. Is this the best shape for this data?

Whether this is the "best" shape for the data is subjective, but I would argue that this does not constitute "tidy" data. A good definition of a single observation for this data would be: the amount of coffee in a given carafe at a given hour. By this definition, we should have columns: `['hour', 'carafe', 'coffee_level']`

### 3. Cake Recipes

#### 3a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [20]:
url = get_db_url('tidy_data')
sql = '''
SELECT *
  FROM cake_recipes
'''
cake_recipes = pd.read_sql(sql, url)

In [21]:
df = cake_recipes

In [22]:
df.head(3)

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541


In [23]:
df['recipe'] = df['recipe:position'].str.split(':').str[0]
df['position'] = df['recipe:position'].str.split(':').str[1]
df = df.drop(columns='recipe:position')

In [24]:
df = df.melt(id_vars=['recipe', 'position'], var_name='oven_temp', value_name='score')

In [25]:
df = df.rename(columns={'position': 'oven_rack_position'})

In [26]:
df.head(3)

,recipe,oven_rack_position,oven_temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320


#### 3c. Which recipe, on average, is the best?

In [27]:
df.groupby(by='recipe').score.mean().idxmax()

'b'

#### 3d. Which oven temperature, on average, produces the best results?

In [28]:
df.groupby(by='oven_temp').score.mean().idxmax()

'275'

#### 3e. Which combination of recipe, rack position, and temperature gives the best result?


In [29]:
df[df.score == df.score.max()]

,recipe,oven_rack_position,oven_temp,score
26,b,bottom,300,99.248541


## 4. Bonus

#### explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.


In [30]:
url = get_db_url('tidy_data')
sql = '''
SHOW TABLES
'''
pd.read_sql(sql, url)

,Tables_in_tidy_data
0,attendance
1,billboard
2,cake_recipes
3,coffee_levels
4,gapminder1
5,gapminder2
6,melted
7,pew
8,sales
9,students


### billboard data

In [31]:
url = get_db_url('tidy_data')
sql = '''
SELECT * 
  FROM billboard
'''
billboard = pd.read_sql(sql, url)

In [32]:
df = billboard
df.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,wk6,wk7,wk8,wk9,wk10,wk11,wk12,wk13,wk14,wk15,wk16,wk17,wk18,wk19,wk20,wk21,wk22,wk23,wk24,wk25,wk26,wk27,wk28,wk29,wk30,wk31,wk32,wk33,wk34,wk35,wk36,wk37,wk38,wk39,wk40,wk41,wk42,wk43,wk44,wk45,wk46,wk47,wk48,wk49,wk50,wk51,wk52,wk53,wk54,wk55,wk56,wk57,wk58,wk59,wk60,wk61,wk62,wk63,wk64,wk65,wk66,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,94.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,57.0,54.0,53.0,51.0,51.0,51.0,51.0,47.0,44.0,38.0,28.0,22.0,18.0,18.0,14.0,12.0,7.0,6.0,6.0,6.0,5.0,5.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,5.0,5.0,9.0,9.0,15.0,14.0,13.0,14.0,16.0,17.0,21.0,22.0,24.0,28.0,33.0,42.0,42.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,65.0,55.0,59.0,62.0,61.0,61.0,59.0,61.0,66.0,72.0,76.0,75.0,67.0,73.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,31.0,36.0,49.0,53.0,57.0,64.0,70.0,75.0,76.0,78.0,85.0,92.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None


In [33]:
df = df.melt(id_vars=['year', 'artist', 'track', 'time', 'date.entered'], 
             var_name='week', 
             value_name='chart_position')

In [34]:
df['week'] = df.week.str[2:].astype(int)

In [35]:
df = df[['track', 'artist', 'time', 'year', 'week', 'chart_position', 'date.entered']]

In [36]:
df = df.rename(columns={'track': 'track_name',
                        'time': 'track_duration',
                        'date.entered': 'date_entered'})

In [37]:
df.head()

,track_name,artist,track_duration,year,week,chart_position,date_entered
0,Baby Don't Cry (Keep...,2 Pac,4:22,2000,1,87,2000-02-26
1,The Hardest Part Of ...,2Ge+her,3:15,2000,1,91,2000-09-02
2,Kryptonite,3 Doors Down,3:53,2000,1,81,2000-04-08
3,Loser,3 Doors Down,4:24,2000,1,76,2000-10-21
4,Wobble Wobble,504 Boyz,3:35,2000,1,57,2000-04-15


## melted data

In [38]:
url = get_db_url('tidy_data')
sql = '''
SELECT * 
  FROM melted
'''
melted = pd.read_sql(sql, url)

In [39]:
df = melted
df.head(10)

,date,variable,value
0,2000-01-03,A,-0.456694
1,2000-01-03,B,-0.446287
2,2000-01-03,C,-0.839698
3,2000-01-03,D,0.963339
4,2000-01-04,A,-1.758507
5,2000-01-04,B,-0.585874
6,2000-01-04,C,1.824410
7,2000-01-04,D,0.247688
8,2000-01-05,A,-1.233451
9,2000-01-05,B,0.483213


In [40]:
df = df.pivot_table(index='date',columns='variable').reset_index()
df.columns.names = [None, None]

In [41]:
df.head(3)

date     value                              
                      A         B         C         D
0  2000-01-03 -0.456694 -0.446287 -0.839698  0.963339
1  2000-01-04 -1.758507 -0.585874  1.824410  0.247688
2  2000-01-05 -1.233451  0.483213  0.928141 -0.547720

## pew data

In [42]:
url = get_db_url('tidy_data')
sql = '''
SELECT * 
  FROM pew
'''
pew = pd.read_sql(sql, url)

In [43]:
df = pew
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [44]:
df = df.melt(id_vars='religion', var_name='income_bracket', value_name='count')

In [45]:
df.head()

,religion,income_bracket,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


In [46]:
df

,religion,income_bracket,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8
